In [37]:
import os
import random
import requests

from bs4 import BeautifulSoup as bs4
from datetime import datetime, time, timedelta
from time import sleep
from github import Github
# Authentication is defined via github.Auth
from github import Auth

# using an access token
auth = Auth.Token(os.getenv('GITHUB_TOKEN'))

# Public Web Github
g = Github(auth=auth)

# setup Github
REPO_NAME = os.getenv('REPO_NAME')
repo = g.get_user().get_repo(REPO_NAME)

print('repo: ', REPO_NAME)
print('token: ', g)
print('user: ', repo)

# setup variables
push_min = int(os.getenv('PUSH_MIN', '1'))
push_max = int(os.getenv('PUSH_MAX', '5'))
prob = int(os.getenv('PROB', '30'))
prob_we = int(os.getenv('PROB_WE', '10'))
start = int(os.getenv('START', '9'))
stop = int(os.getenv('STOP', '17'))

URL = os.environ.get('URL')

repo:  githuber
token:  <github.MainClass.Github object at 0x7f4a757550f0>
user:  Repository(full_name="d-gilles/githuber")


In [2]:
# scrape the book
def scrape():
    # get the url
    if URL is not None:
        page = requests.get(URL)
        # parse the html
        soup = bs4(page.text, 'html.parser')
        # find the elements
        elements = soup.find_all(['h2', 'p'])
        # return the text
        lines = [element.text for element in elements]
        book = '\n'.join(lines[0:2] + lines[3:9])
        return book
    else:
        return ''
    

In [32]:
def push_to_github():
    # get the book
    book = scrape()
    filename = 'the_book.txt'

    # pull the file from github
    contents = repo.get_contents(filename, ref='master')
    print('contents: ', contents)
    print('lenth of book at github: ', len(contents.decoded_content.decode()))
    if len(contents.decoded_content.decode()) > 1:
        
        print('book at github: ', contents.decoded_content.decode())
        lob = [x for x in contents.decoded_content.decode().split('\n')]
    else:
        lob = []    
    print ('book at github: ', lob)
    l1 = len(lob)
    print('lines of book at github: ', l1)
    line_to_append = book.split('\n')[l1:l1+1]
    print('line to append: ', line_to_append)
    lob += line_to_append
    print('lenth of book after append: ', len(lob))

    #repo.update_file(contents.path, f'add line nr.: {l1+1}', '\n'.join(lob), contents.sha, branch='master')
    print(f'pushed line nr.: {l1+1}')
    return lob

In [34]:
push_to_github()

contents:  ContentFile(path="the_book.txt")
lenth of book at github:  40
book at github:  Le Petit Prince
Antoine de Saint-Exupéry
book at github:  ['Le Petit Prince', 'Antoine de Saint-Exupéry']
lenth of book at github:  2
line to append:  ['DEDICACE']
lenth of book after append:  3
pushed line nr.: 3


['Le Petit Prince', 'Antoine de Saint-Exupéry', 'DEDICACE']

In [79]:
def schedule_pushes(push_min=1, push_max=12, prob=20, prob_we=85, start=11, stop=20):
    
    is_weekend = datetime.today().weekday() >= 5
    mood = random.random()
    lazy = random.random()
    
    print('weekend: ', is_weekend)
    print('threadhold: ', prob_we/100)
    print('mood is: ', mood, (mood < prob_we/100) )
    print('lazy? : ', lazy, (lazy < prob/100))
    
    if (is_weekend and (mood < prob_we/100)) or ((not is_weekend) and (lazy < prob/100)):
        print('do nothing today')
        return 0
    else:
        num_pushes = random.randint(push_min, push_max)
        print(f"Will push {num_pushes} times today starting at {start}am and ending at {stop}pm")
        
        x = datetime.now() + timedelta(minutes=1)
        y = datetime.now() + timedelta(hours=stop - start)
        n = (stop - start) / (num_pushes)
        
        interval_in_secounds = n*60*60
        jitter = interval_in_secounds/2*0.9
        
        print('start at', x)
        print('stop at', y)
        print('interval', n)
        print('interval in seconds = ', interval_in_secounds)
        print('jitter = ', interval_in_secounds/2*0.9)
        
        return num_pushes
    
    

In [80]:
schedule_pushes(push_min, push_max, prob, prob_we, start, stop)

weekend:  False
threadhold:  0.85
mood is:  0.5553249334452173 True
lazy? :  0.11488693547720175 True
do nothing today


0